In [1]:
"""
Creamos el dataset.
1.- Extraemos frames de un video
2.- Detectamos rostros en los frames
3.- Cramos las mascaras de los rostros
"""

'\nCreamos el dataset.\n1.- Extraemos frames de un video\n2.- Detectamos rostros en los frames\n3.- Cramos las mascaras de los rostros\n'

In [7]:
import cv2
import dlib
import numpy as np
import os
from os.path import isfile, isdir, exists

ModuleNotFoundError: No module named 'dlib'

In [3]:
"""
Referencia: https://github.com/gallardorafael/edge2art
Extrae cada frame de un video y guardas las imagenes
"""
EVERY_FRAMES = 5
def extraer_frames(file_input, dir_output):
        # Validamos la existencia del video
        if isfile(file_input):
            video_path = file_input
        else:
            print("No existe el video.")
            return
        
        #Definición del CODEC
        cap = cv2.VideoCapture(file_input)
        if not exists(dir_output):
            os.mkdir(dir_output)

        # OpenCV
        contador = 0
        while cap.isOpened():
            # Obtenemos el frame.
            ret, frame = cap.read()
            if ret == True:
                if contador % EVERY_FRAMES == 0:
                    data_path = dir_output + "/" + str(contador) + ".png"
                    print("Extrayendo frame ", contador, " en ", data_path)
                    cv2.imwrite(data_path, frame)
                contador += 1
            else:
                break
        cap.release()

In [4]:
"""
OpenCV’s deep learning face detector esta basado en el
framework Single Shot Detector (SSD) con base en una red
ResNet 
"""

# Para el recorte
CROP_SIZE = 256

# Cargamos modelo de la red
path_model="./deploy.proto.txt"
path_weigths="./res10_300x300_ssd_iter_140000.caffemodel"
net = cv2.dnn.readNetFromCaffe(path_model, path_weigths)
MIN_CONFIDENCE = 0.8 # Probabilidad minima para la detección

def detectar_rostro(image):
    # Cargamos la imagen y construimos una entraba para la red
    # reescalamos a 300x300 y la normalizamos
    (h, w) = image.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    
    # Pasamos la entrada a la red y obtenemos las detecciones y 
    # predicciones
    net.setInput(blob)
    detections = net.forward()

    # Iteramos sobre las detecciones
    for i in range(0, detections.shape[2]):
        # Extraemos la probabilidad asociada a una prediccion
        confidence = detections[0, 0, i, 2]

        # Comparamos las probabilidades y verificamos que la actual 
        # sea mayor a la minima, para asegurar que sí es correcta
        # la detección
        if confidence > MIN_CONFIDENCE:
            # Calculamos las coordenadas (x, y) para el rectangulo que
            # enmarque el rostro
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            y = startY - 10 if startY - 10 > 10 else startY + 10
            distanceX = abs(endX-startX)
            distanceY = abs(endY-startY)
            startY -= distanceY * 0.2
            endY += distanceY * 0.1
            endX += distanceX * 0.5
            startX -= distanceX * 0.5
            startX = int(max(startX,0))
            startY = int(max(startY,0))
            endX = int(min(endX,w))
            endY = int(min(endY,h))
            image = image[startY:endY,startX:endX,:]
            return cv2.resize(image,(CROP_SIZE, CROP_SIZE))

def detectar_rostros_en_frames(dir_input, dir_output):
    count = 0
    # Se crea el directorio de salida
    if not isdir(dir_output):
        os.mkdir(dir_output)
    for artwork_name in os.listdir(dir_input):
        count += 1
        artwork_path = os.path.join(dir_input, artwork_name)
        target_path = os.path.join(dir_output, artwork_name)
        print("Detectando rostro ", count, " en ", target_path)
        image = cv2.imread(artwork_path)
        image = detectar_rostro(image)
        cv2.imwrite(target_path, image)

In [5]:
file_input = "./video1.mp4"
dir_output =  "./tagged_train"
extraer_frames(file_input, dir_output)

file_input = "./video2.mp4"
dir_output =  "./tagged_test"
extraer_frames(file_input, dir_output)

Extrayendo frame  0  en  ./tagged_train/0.png
Extrayendo frame  5  en  ./tagged_train/5.png
Extrayendo frame  10  en  ./tagged_train/10.png
Extrayendo frame  15  en  ./tagged_train/15.png
Extrayendo frame  20  en  ./tagged_train/20.png
Extrayendo frame  25  en  ./tagged_train/25.png
Extrayendo frame  30  en  ./tagged_train/30.png
Extrayendo frame  35  en  ./tagged_train/35.png
Extrayendo frame  40  en  ./tagged_train/40.png
Extrayendo frame  45  en  ./tagged_train/45.png
Extrayendo frame  50  en  ./tagged_train/50.png
Extrayendo frame  55  en  ./tagged_train/55.png
Extrayendo frame  60  en  ./tagged_train/60.png
Extrayendo frame  65  en  ./tagged_train/65.png
Extrayendo frame  70  en  ./tagged_train/70.png
Extrayendo frame  75  en  ./tagged_train/75.png
Extrayendo frame  80  en  ./tagged_train/80.png
Extrayendo frame  85  en  ./tagged_train/85.png
Extrayendo frame  90  en  ./tagged_train/90.png
Extrayendo frame  95  en  ./tagged_train/95.png
Extrayendo frame  100  en  ./tagged_train/10

Extrayendo frame  825  en  ./tagged_train/825.png
Extrayendo frame  830  en  ./tagged_train/830.png
Extrayendo frame  835  en  ./tagged_train/835.png
Extrayendo frame  840  en  ./tagged_train/840.png
Extrayendo frame  845  en  ./tagged_train/845.png
Extrayendo frame  850  en  ./tagged_train/850.png
Extrayendo frame  855  en  ./tagged_train/855.png
Extrayendo frame  860  en  ./tagged_train/860.png
Extrayendo frame  865  en  ./tagged_train/865.png
Extrayendo frame  870  en  ./tagged_train/870.png
Extrayendo frame  875  en  ./tagged_train/875.png
Extrayendo frame  880  en  ./tagged_train/880.png
Extrayendo frame  885  en  ./tagged_train/885.png
Extrayendo frame  890  en  ./tagged_train/890.png
Extrayendo frame  895  en  ./tagged_train/895.png
Extrayendo frame  900  en  ./tagged_train/900.png
Extrayendo frame  905  en  ./tagged_train/905.png
Extrayendo frame  910  en  ./tagged_train/910.png
Extrayendo frame  915  en  ./tagged_train/915.png
Extrayendo frame  920  en  ./tagged_train/920.png


Extrayendo frame  1620  en  ./tagged_train/1620.png
Extrayendo frame  1625  en  ./tagged_train/1625.png
Extrayendo frame  1630  en  ./tagged_train/1630.png
Extrayendo frame  1635  en  ./tagged_train/1635.png
Extrayendo frame  1640  en  ./tagged_train/1640.png
Extrayendo frame  1645  en  ./tagged_train/1645.png
Extrayendo frame  1650  en  ./tagged_train/1650.png
Extrayendo frame  1655  en  ./tagged_train/1655.png
Extrayendo frame  1660  en  ./tagged_train/1660.png
Extrayendo frame  1665  en  ./tagged_train/1665.png
Extrayendo frame  1670  en  ./tagged_train/1670.png
Extrayendo frame  1675  en  ./tagged_train/1675.png
Extrayendo frame  1680  en  ./tagged_train/1680.png
Extrayendo frame  1685  en  ./tagged_train/1685.png
Extrayendo frame  1690  en  ./tagged_train/1690.png
Extrayendo frame  1695  en  ./tagged_train/1695.png
Extrayendo frame  1700  en  ./tagged_train/1700.png
Extrayendo frame  1705  en  ./tagged_train/1705.png
Extrayendo frame  1710  en  ./tagged_train/1710.png
Extrayendo f

Extrayendo frame  620  en  ./tagged_test/620.png
Extrayendo frame  625  en  ./tagged_test/625.png
Extrayendo frame  630  en  ./tagged_test/630.png
Extrayendo frame  635  en  ./tagged_test/635.png
Extrayendo frame  640  en  ./tagged_test/640.png
Extrayendo frame  645  en  ./tagged_test/645.png
Extrayendo frame  650  en  ./tagged_test/650.png
Extrayendo frame  655  en  ./tagged_test/655.png
Extrayendo frame  660  en  ./tagged_test/660.png
Extrayendo frame  665  en  ./tagged_test/665.png
Extrayendo frame  670  en  ./tagged_test/670.png
Extrayendo frame  675  en  ./tagged_test/675.png
Extrayendo frame  680  en  ./tagged_test/680.png
Extrayendo frame  685  en  ./tagged_test/685.png
Extrayendo frame  690  en  ./tagged_test/690.png
Extrayendo frame  695  en  ./tagged_test/695.png
Extrayendo frame  700  en  ./tagged_test/700.png
Extrayendo frame  705  en  ./tagged_test/705.png
Extrayendo frame  710  en  ./tagged_test/710.png
Extrayendo frame  715  en  ./tagged_test/715.png
Extrayendo frame  72

Extrayendo frame  1445  en  ./tagged_test/1445.png
Extrayendo frame  1450  en  ./tagged_test/1450.png
Extrayendo frame  1455  en  ./tagged_test/1455.png
Extrayendo frame  1460  en  ./tagged_test/1460.png
Extrayendo frame  1465  en  ./tagged_test/1465.png
Extrayendo frame  1470  en  ./tagged_test/1470.png
Extrayendo frame  1475  en  ./tagged_test/1475.png
Extrayendo frame  1480  en  ./tagged_test/1480.png
Extrayendo frame  1485  en  ./tagged_test/1485.png
Extrayendo frame  1490  en  ./tagged_test/1490.png
Extrayendo frame  1495  en  ./tagged_test/1495.png
Extrayendo frame  1500  en  ./tagged_test/1500.png
Extrayendo frame  1505  en  ./tagged_test/1505.png
Extrayendo frame  1510  en  ./tagged_test/1510.png
Extrayendo frame  1515  en  ./tagged_test/1515.png
Extrayendo frame  1520  en  ./tagged_test/1520.png
Extrayendo frame  1525  en  ./tagged_test/1525.png
Extrayendo frame  1530  en  ./tagged_test/1530.png
Extrayendo frame  1535  en  ./tagged_test/1535.png
Extrayendo frame  1540  en  ./t

In [6]:
dir_input = "./tagged_train"
dir_output =  "./input_train"
detectar_rostros_en_frames(dir_input, dir_output)

dir_input = "./tagged_test"
dir_output =  "./input_test"
detectar_rostros_en_frames(dir_input, dir_output)

Detectando rostro  1  en  ./input_train/210.png
Detectando rostro  2  en  ./input_train/525.png
Detectando rostro  3  en  ./input_train/295.png
Detectando rostro  4  en  ./input_train/575.png
Detectando rostro  5  en  ./input_train/935.png
Detectando rostro  6  en  ./input_train/355.png
Detectando rostro  7  en  ./input_train/540.png
Detectando rostro  8  en  ./input_train/1080.png
Detectando rostro  9  en  ./input_train/1315.png
Detectando rostro  10  en  ./input_train/690.png
Detectando rostro  11  en  ./input_train/565.png
Detectando rostro  12  en  ./input_train/220.png
Detectando rostro  13  en  ./input_train/1610.png
Detectando rostro  14  en  ./input_train/1220.png
Detectando rostro  15  en  ./input_train/570.png
Detectando rostro  16  en  ./input_train/1155.png
Detectando rostro  17  en  ./input_train/1415.png
Detectando rostro  18  en  ./input_train/245.png
Detectando rostro  19  en  ./input_train/620.png
Detectando rostro  20  en  ./input_train/990.png
Detectando rostro  21  

Detectando rostro  167  en  ./input_train/290.png
Detectando rostro  168  en  ./input_train/1695.png
Detectando rostro  169  en  ./input_train/1630.png
Detectando rostro  170  en  ./input_train/270.png
Detectando rostro  171  en  ./input_train/1055.png
Detectando rostro  172  en  ./input_train/655.png
Detectando rostro  173  en  ./input_train/1600.png
Detectando rostro  174  en  ./input_train/405.png
Detectando rostro  175  en  ./input_train/365.png
Detectando rostro  176  en  ./input_train/1680.png
Detectando rostro  177  en  ./input_train/1370.png
Detectando rostro  178  en  ./input_train/910.png
Detectando rostro  179  en  ./input_train/370.png
Detectando rostro  180  en  ./input_train/865.png
Detectando rostro  181  en  ./input_train/515.png
Detectando rostro  182  en  ./input_train/1690.png
Detectando rostro  183  en  ./input_train/725.png
Detectando rostro  184  en  ./input_train/390.png
Detectando rostro  185  en  ./input_train/1635.png
Detectando rostro  186  en  ./input_train/

Detectando rostro  331  en  ./input_train/600.png
Detectando rostro  332  en  ./input_train/1720.png
Detectando rostro  333  en  ./input_train/1065.png
Detectando rostro  334  en  ./input_train/215.png
Detectando rostro  335  en  ./input_train/1090.png
Detectando rostro  336  en  ./input_train/1235.png
Detectando rostro  337  en  ./input_train/160.png
Detectando rostro  338  en  ./input_train/25.png
Detectando rostro  339  en  ./input_train/660.png
Detectando rostro  340  en  ./input_train/1770.png
Detectando rostro  341  en  ./input_train/830.png
Detectando rostro  342  en  ./input_train/1115.png
Detectando rostro  343  en  ./input_train/1520.png
Detectando rostro  344  en  ./input_train/1100.png
Detectando rostro  345  en  ./input_train/775.png
Detectando rostro  346  en  ./input_train/1830.png
Detectando rostro  347  en  ./input_train/165.png
Detectando rostro  348  en  ./input_train/1435.png
Detectando rostro  349  en  ./input_train/1015.png
Detectando rostro  350  en  ./input_trai

Detectando rostro  131  en  ./input_test/1595.png
Detectando rostro  132  en  ./input_test/765.png
Detectando rostro  133  en  ./input_test/790.png
Detectando rostro  134  en  ./input_test/485.png
Detectando rostro  135  en  ./input_test/1145.png
Detectando rostro  136  en  ./input_test/580.png
Detectando rostro  137  en  ./input_test/840.png
Detectando rostro  138  en  ./input_test/880.png
Detectando rostro  139  en  ./input_test/105.png
Detectando rostro  140  en  ./input_test/1710.png
Detectando rostro  141  en  ./input_test/1450.png
Detectando rostro  142  en  ./input_test/680.png
Detectando rostro  143  en  ./input_test/555.png
Detectando rostro  144  en  ./input_test/300.png
Detectando rostro  145  en  ./input_test/1325.png
Detectando rostro  146  en  ./input_test/1285.png
Detectando rostro  147  en  ./input_test/1125.png
Detectando rostro  148  en  ./input_test/1590.png
Detectando rostro  149  en  ./input_test/780.png
Detectando rostro  150  en  ./input_test/445.png
Detectando r

Detectando rostro  298  en  ./input_test/265.png
Detectando rostro  299  en  ./input_test/730.png
Detectando rostro  300  en  ./input_test/1685.png
Detectando rostro  301  en  ./input_test/930.png
Detectando rostro  302  en  ./input_test/1105.png
Detectando rostro  303  en  ./input_test/820.png
Detectando rostro  304  en  ./input_test/1020.png
Detectando rostro  305  en  ./input_test/1460.png
Detectando rostro  306  en  ./input_test/1645.png
Detectando rostro  307  en  ./input_test/190.png
Detectando rostro  308  en  ./input_test/1725.png
Detectando rostro  309  en  ./input_test/885.png
Detectando rostro  310  en  ./input_test/1265.png
Detectando rostro  311  en  ./input_test/500.png
Detectando rostro  312  en  ./input_test/395.png
Detectando rostro  313  en  ./input_test/640.png
Detectando rostro  314  en  ./input_test/970.png
Detectando rostro  315  en  ./input_test/915.png
Detectando rostro  316  en  ./input_test/600.png
Detectando rostro  317  en  ./input_test/1720.png
Detectando r